### 중복 불가 하니까 [1 에서 사용된 1이 1.에서는 사용 안되는 문제 발생

In [41]:
import cv2
import easyocr
import re
import glob
import os

# 이미지 파일들이 있는 디렉토리 경로
directory = r'C:\Users\hana9\preprocessing\a_kor_test\2015_G1_11_KOR\cropped'

# 결과 이미지를 저장할 디렉토리 경로
output_directory = r'C:\Users\hana9\preprocessing\output_images_kor_test'

# 해당 디렉토리 내의 모든 png 파일 찾기
png_files = glob.glob(os.path.join(directory, '*.png'))

def extract_sort_key(filename):
    # 파일명에서 left 또는 right 부분과 숫자 부분을 추출하여 정렬 키를 반환하는 함수
    match = re.match(r'.*_(left|right)__page(\d+)\.png', filename)
    if match:
        prefix = match.group(1)
        number = int(match.group(2))
        return (number, prefix)  # 숫자를 먼저 비교하고, 그 다음에 left/right를 비교하여 정렬

    # 매칭되지 않는 경우는 마지막으로 처리
    return (float('inf'), filename)

# 파일을 정렬합니다.
sorted_files = sorted(png_files, key=extract_sort_key)

# EasyOCR을 사용하여 텍스트 인식
reader = easyocr.Reader(['ko', 'en'])

# 정규 표현식으로 매칭되는 패턴들 정의
pattern1 = re.compile(r"\b([0-9]|[1-3][0-9]|4[0-5])\.\s*")
pattern2 = re.compile(r'^(?:\d{1,2}\s*<|\d{1,2}\s*윗글)')
pattern3 = re.compile(r'\[(\d+)\s*~\s*(\d+)\]|\[\d+\s*\b')

patterns = [pattern1, pattern2, pattern3]

# 모든 가능한 숫자 리스트 (1~45)
all_numbers = set(range(1, 46))

# 이미 사용된 숫자를 저장하는 집합
used_numbers_set = set()

# 각 이미지에 대해 처리
for idx, f_name in enumerate(sorted_files):
    # left_page0마다 숫자 리스트 리셋
    if 'left__page0' in f_name:
        all_numbers = set(range(1, 46))
    # 이미지 파일 읽기
    img = cv2.imread(f_name)

    # EasyOCR을 사용하여 텍스트 인식
    results = reader.readtext(f_name)

    # 정규 표현식으로 매칭되는 패턴 찾기
    def find_patterns(results, patterns):
        found_patterns = []
        for pattern in patterns:
            for (bbox, text, prob) in results:
                if re.match(pattern, text):
                    found_patterns.append((bbox, text, prob))
        return found_patterns

    # 패턴 찾기
    found_patterns = find_patterns(results, patterns)

    if found_patterns:
        # 패턴 출력 (디버깅용)
        print("Found patterns:", found_patterns)

        # y 좌표로 정렬
        found_patterns.sort(key=lambda b: b[0][0][1])

        # 정렬된 결과 출력
        print("Sorted patterns by y-coordinate:")
        for pattern in found_patterns:
            print(pattern)

        # 첫 번째 패턴의 상단 y 좌표 구하기
        first_y_min = int(found_patterns[0][0][0][1])

        # 첫 번째 패턴이 y=50보다 큰 경우, 원본 이미지의 상단부터 첫 번째 패턴까지 자르기
        if first_y_min > 50:
            cropped_img = img[0:first_y_min:, :]

            # 결과 이미지 저장
            output_file = os.path.join(output_directory, f"{idx}_top_to_first.png")
            cv2.imwrite(output_file, cropped_img)

        # 패턴이 있는 텍스트의 내용과 위치 정보를 활용하여 처리
        for i, (bbox1, text1, _) in enumerate(found_patterns):
            if i < len(found_patterns) - 1:
                bbox2, text2, _ = found_patterns[i + 1]

                # 첫 번째 패턴의 상단과 두 번째 패턴의 상단 사이의 이미지 부분 자르기
                x_min = min(int(bbox1[0][0]), int(bbox2[0][0]))
                y_min = min(int(bbox1[0][1]), int(bbox2[0][1]))
                x_max = max(int(bbox1[2][0]), int(bbox2[2][0]))
                y_max = max(int(bbox1[0][1]), int(bbox2[0][1]))

                cropped_img = img[y_min:y_max, x_min:]

            # 결과 이미지 저장
            # output_file = os.path.join(output_directory, f"{idx}_{i}.png")
            # cv2.imwrite(output_file, cropped_img)

                # 결과 이미지 저장 (found_patterns에서 숫자 추출하여 파일명으로 사용)
                pattern_number = re.findall(r'\d+', text1)[0]  # found_patterns에서 숫자 추출
                if pattern_number:
                    pattern_number = int(pattern_number[0])
                    if pattern_number not in used_numbers_set:
                        # 결과 이미지 저장
                        output_file = os.path.join(output_directory, f"{idx}_{pattern_number}_{i}.png")
                        cv2.imwrite(output_file, cropped_img)

                        # 사용된 숫자 집합에 추가
                        used_numbers_set.add(pattern_number)

        # 마지막 패턴의 상단부터 원본 이미지의 끝까지 자르기
        last_bbox, last_text, _ = found_patterns[-1]
        x_min, y_min = int(last_bbox[0][0]), int(last_bbox[0][1])
        # y_min = int(last_bbox[0][1])
        cropped_img = img[y_min:-300, x_min:]

        # 패턴에서 숫자 추출
        pattern_numbers = re.findall(r'\d+', last_text) # found_patterns에서 숫자 추출
        if pattern_numbers:
            pattern_number = int(pattern_numbers[0])
            if pattern_number not in used_numbers_set:
                # 결과 이미지 저장
                output_file = os.path.join(output_directory, f"{idx}_{pattern_number}_{i}.png")
                cv2.imwrite(output_file, cropped_img)
                
                # 사용된 숫자 집합에 추가
                used_numbers_set.add(pattern_number)

    else:
        # 패턴이 발견되지 않으면 전체 이미지를 저장
        height, width, _ = img.shape
        cropped_img = img[0:height, 0:width]

        # 결과 이미지 저장 (파일명을 임의로 지정)
        output_file = os.path.join(output_directory, f"whole_image_{idx}.png")
        cv2.imwrite(output_file, cropped_img)
        
    # 현재까지 사용된 숫자 리스트 출력
    print("Used numbers so far:", sorted(used_numbers_set))
    print("Remaining numbers:", sorted(all_numbers))

# 모든 이미지 처리 완료 후에 남은 숫자 리스트 출력 및 파일로 저장
remaining_numbers = sorted(all_numbers - used_numbers_set)
print("Remaining numbers list:", remaining_numbers)

# 결과 텍스트 파일에 남은 숫자 리스트 저장
result_filename = "remaining_numbers.txt"
result_filepath = os.path.join(output_directory, result_filename)

with open(result_filepath, 'w') as f:
    for num in remaining_numbers:
        f.write(f"{num}\n")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Found patterns: [([[232, 27], [265, 27], [265, 65], [232, 65]], '[1', 0.9999710030913613)]
Sorted patterns by y-coordinate:
([[232, 27], [265, 27], [265, 65], [232, 65]], '[1', 0.9999710030913613)
Used numbers so far: [1]
Found patterns: [([[15, 27], [927, 27], [927, 78], [15, 78]], '1. 토론 참가자들의 말하기 방식에 대한 설명으로 적절하지 않은 것은?', 0.6311627868636461), ([[15, 603], [931, 603], [931, 652], [15, 652]], '2. <보기> 의 자료틀 토론에   활용할 수 있는 입장과 방안올  짝지은', 0.26223759210126185), ([[16, 2107], [553, 2107], [553, 2151], [16, 2151]], '3. 양추이 공통으로 인정하고 있는 것은?', 0.4070365368944325)]
Sorted patterns by y-coordinate:
([[15, 27], [927, 27], [927, 78], [15, 78]], '1. 토론 참가자들의 말하기 방식에 대한 설명으로 적절하지 않은 것은?', 0.6311627868636461)
([[15, 603], [931, 603], [931, 652], [15, 652]], '2. <보기> 의 자료틀 토론에   활용할 수 있는 입장과 방안올  짝지은', 0.26223759210126185)
([[16, 2107], [553, 2107], [553, 2151], [16, 2151]], '3. 양추이 공통으로 인정하고 있는 것은?', 0.4070365368944325)
Used numbers so far: [1, 2, 3]


KeyboardInterrupt: 

In [44]:
import cv2
import easyocr
import re
import glob
import os

# 이미지 파일들이 있는 디렉토리 경로
directory = r'C:\Users\hana9\preprocessing\a_kor_test\2015_G1_11_KOR\cropped'

# 결과 이미지를 저장할 디렉토리 경로
output_directory = r'C:\Users\hana9\preprocessing\output_images_kor_test'

# 해당 디렉토리 내의 모든 png 파일 찾기
png_files = glob.glob(os.path.join(directory, '*.png'))

def extract_sort_key(filename):
    # 파일명에서 left 또는 right 부분과 숫자 부분을 추출하여 정렬 키를 반환하는 함수
    match = re.match(r'.*_(left|right)__page(\d+)\.png', filename)
    if match:
        prefix = match.group(1)
        number = int(match.group(2))
        return (number, prefix)  # 숫자를 먼저 비교하고, 그 다음에 left/right를 비교하여 정렬

    # 매칭되지 않는 경우는 마지막으로 처리
    return (float('inf'), filename)

# 파일을 정렬합니다.
sorted_files = sorted(png_files, key=extract_sort_key)

# EasyOCR을 사용하여 텍스트 인식
reader = easyocr.Reader(['ko', 'en'])

# 정규 표현식으로 매칭되는 패턴들 정의
pattern1 = re.compile(r"\b([0-9]|[1-3][0-9]|4[0-5])\.\s*")
pattern2 = re.compile(r'^(?:\d{1,2}\s*<|\d{1,2}\s*윗글)')
pattern3 = re.compile(r'\[(\d+)\s*~\s*(\d+)\]|\[\d+\s*\b')

patterns = [pattern1, pattern2, pattern3]

# 각 이미지에 대해 처리
for idx, f_name in enumerate(sorted_files):
    # 이미지 파일 읽기
    img = cv2.imread(f_name)

    # EasyOCR을 사용하여 텍스트 인식
    results = reader.readtext(f_name)

    # 정규 표현식으로 매칭되는 패턴 찾기
    def find_patterns(results, patterns):
        found_patterns = []
        for pattern in patterns:
            for (bbox, text, prob) in results:
                if re.match(pattern, text):
                    found_patterns.append((bbox, text, prob))
        return found_patterns

    # 패턴 찾기
    found_patterns = find_patterns(results, patterns)

    if found_patterns:
        # 패턴 출력 (디버깅용)
        print("Found patterns:", found_patterns)

        # y 좌표로 정렬
        found_patterns.sort(key=lambda b: b[0][0][1])

        # 정렬된 결과 출력
        print("Sorted patterns by y-coordinate:")
        for pattern in found_patterns:
            print(pattern)

        # 첫 번째 패턴의 상단 y 좌표 구하기
        first_y_min = int(found_patterns[0][0][0][1])

        # 첫 번째 패턴이 y=50보다 큰 경우, 원본 이미지의 상단부터 첫 번째 패턴까지 자르기
        if first_y_min > 50:
            cropped_img = img[0:first_y_min:, :]

            # 결과 이미지 저장
            output_file = os.path.join(output_directory, f"{idx}_top_to_first.png")
            cv2.imwrite(output_file, cropped_img)

        # 패턴이 있는 텍스트의 내용과 위치 정보를 활용하여 처리
        for i, (bbox1, text1, _) in enumerate(found_patterns):
            if i < len(found_patterns) - 1:
                bbox2, text2, _ = found_patterns[i + 1]

                # 첫 번째 패턴의 상단과 두 번째 패턴의 상단 사이의 이미지 부분 자르기
                x_min = min(int(bbox1[0][0]), int(bbox2[0][0]))
                y_min = min(int(bbox1[0][1]), int(bbox2[0][1]))
                x_max = max(int(bbox1[2][0]), int(bbox2[2][0]))
                y_max = max(int(bbox1[0][1]), int(bbox2[0][1]))

                cropped_img = img[y_min:y_max, x_min:]

            # 결과 이미지 저장
            # output_file = os.path.join(output_directory, f"{idx}_{i}.png")
            # cv2.imwrite(output_file, cropped_img)

                # 결과 이미지 저장 (found_patterns에서 숫자 추출하여 파일명으로 사용)
                pattern_number = re.findall(r'\d+', text1)[0]  # found_patterns에서 숫자 추출
                output_file = os.path.join(output_directory, f"{idx}_{pattern_number}_{i}.png")
                cv2.imwrite(output_file, cropped_img)

        # 마지막 패턴의 상단부터 원본 이미지의 끝까지 자르기
        last_bbox, last_text, _ = found_patterns[-1]
        x_min, y_min = int(last_bbox[0][0]), int(last_bbox[0][1])
        # y_min = int(last_bbox[0][1])
        cropped_img = img[y_min:-320, x_min:]

        # 결과 이미지 저장 (found_patterns에서 숫자 추출하여 파일명으로 사용)
        pattern_number = re.findall(r'\d+', text1)[0]  # found_patterns에서 숫자 추출
        output_file = os.path.join(output_directory, f"{idx}_{pattern_number}_{i}.png")
        cv2.imwrite(output_file, cropped_img)

    else:
        # 패턴이 발견되지 않으면 전체 이미지를 저장
        height, width, _ = img.shape
        cropped_img = img[0:height, 0:width]

        # 결과 이미지 저장 (파일명을 임의로 지정)
        output_file = os.path.join(output_directory, f"whole_image_{idx}.png")
        cv2.imwrite(output_file, cropped_img)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Found patterns: [([[232, 27], [265, 27], [265, 65], [232, 65]], '[1', 0.9999710030913613)]
Sorted patterns by y-coordinate:
([[232, 27], [265, 27], [265, 65], [232, 65]], '[1', 0.9999710030913613)
Found patterns: [([[15, 27], [927, 27], [927, 78], [15, 78]], '1. 토론 참가자들의 말하기 방식에 대한 설명으로 적절하지 않은 것은?', 0.6311627868636461), ([[15, 603], [931, 603], [931, 652], [15, 652]], '2. <보기> 의 자료틀 토론에   활용할 수 있는 입장과 방안올  짝지은', 0.26223759210126185), ([[16, 2107], [553, 2107], [553, 2151], [16, 2151]], '3. 양추이 공통으로 인정하고 있는 것은?', 0.4070365368944325)]
Sorted patterns by y-coordinate:
([[15, 27], [927, 27], [927, 78], [15, 78]], '1. 토론 참가자들의 말하기 방식에 대한 설명으로 적절하지 않은 것은?', 0.6311627868636461)
([[15, 603], [931, 603], [931, 652], [15, 652]], '2. <보기> 의 자료틀 토론에   활용할 수 있는 입장과 방안올  짝지은', 0.26223759210126185)
([[16, 2107], [553, 2107], [553, 2151], [16, 2151]], '3. 양추이 공통으로 인정하고 있는 것은?', 0.4070365368944325)
Found patterns: [([[236, 2192], [269, 2192], [269, 2225], [236, 2225]], '4.', 0.964693610116875), ([[231

KeyboardInterrupt: 

# 정규식으로 잘 찾는지 확인하는 코드

In [ ]:
import cv2
import easyocr
import re
import glob
import os

# 이미지 파일들이 있는 디렉토리 경로
directory = r'C:\Users\hana9\preprocessing\a_kor_test\2015_G1_11_KOR\cropped'

# 결과 이미지를 저장할 디렉토리 경로
output_directory = r'C:\Users\hana9\preprocessing\output_images_kor_test'

# 해당 디렉토리 내의 모든 png 파일 찾기
png_files = glob.glob(os.path.join(directory, '*.png'))

# EasyOCR을 사용하여 텍스트 인식
reader = easyocr.Reader(['ko', 'en'])

def extract_sort_key(filename):
    # 파일명에서 left 또는 right 부분과 숫자 부분을 추출하여 정렬 키를 반환하는 함수
    match = re.match(r'.*_(left|right)__page(\d+)\.png', filename)
    if match:
        prefix = match.group(1)
        number = int(match.group(2))
        return (number, prefix)  # 숫자를 먼저 비교하고, 그 다음에 left/right를 비교하여 정렬

    # 매칭되지 않는 경우는 마지막으로 처리
    return (float('inf'), filename)

# 파일을 정렬합니다.
sorted_files = sorted(png_files, key=extract_sort_key)

# 정규 표현식으로 매칭되는 패턴들 정의
pattern1 = re.compile(r"\b([0-9]|[1-3][0-9]|4[0-5])\.\s*")
pattern2 = re.compile(r'^(?:\d{1,2}\s*<|\d{1,2}\s*윗글)')
pattern3 = re.compile(r'\[(\d+)\s*~\s*(\d+)\]|\[\d+\s*\b')

patterns = [pattern1, pattern2, pattern3]

# 각 이미지에 대해 처리
for idx, f_name in enumerate(sorted_files):
    # 이미지 파일 읽기
    img = cv2.imread(f_name)

    # EasyOCR을 사용하여 텍스트 인식
    results = reader.readtext(f_name)

    # 정규 표현식으로 매칭되는 패턴 찾기
    def find_patterns(results, patterns):
        found_patterns = []
        for pattern in patterns:
            for (bbox, text, prob) in results:
                if re.match(pattern, text):
                    found_patterns.append((bbox, text, prob))
        return found_patterns

    # 패턴 찾기
    found_patterns = find_patterns(results, patterns)

    if found_patterns:
        # 패턴 출력 (디버깅용)
        print("Found patterns:", found_patterns)

        # 패턴이 있는 텍스트의 내용과 위치 정보를 활용하여 처리
        for (bbox, text, prob) in found_patterns:
            print(f"Pattern found at bbox {bbox}: {text}")

    # 추가적인 처리나 저장 코드를 여기에 추가할 수 있습니다.


# 영역 확인 코드

In [ ]:
from matplotlib.pyplot import imshow
from PIL import Image

f_name = r'C:\Users\hana9\preprocessing\a_eng_test\2015_G1_11_ENG\cropped\2015_G1_11_ENG_right__page6.png'

%matplotlib inline
pil_im = Image.open(f_name, 'r')
imshow(pil_im)

import easyocr

# easy-ocr
reader = easyocr.Reader(['ko','en'])
txt = reader.readtext(f_name)
txt